In [12]:
import numpy as np
import os
from embeddings import SentenceEmbeddingDataset, WordEmbeddingManager, create_embedding_dataloader
import utils
import torch

In [2]:
TRAIN_PATH = 'data/raw_train.txt'
train_sentences = utils.load_sentences(TRAIN_PATH)
tokenized_sentences = utils.tokenize_sentences(train_sentences)

In [3]:
WORD2VEC_MODEL_PATH = 'data/word2vec.model'
word2vec_manager = WordEmbeddingManager(WORD2VEC_MODEL_PATH)

Model loaded successfully from data/word2vec.model


In [4]:
dataloader = create_embedding_dataloader(tokenized_sentences, word2vec_manager, seq_length=10, batch_size=4)

In [5]:
first_batch = next(iter(dataloader))

['But', 'I', "'ll", 'acquaint', 'our', 'duteous', 'citizens']
['Who', 'thrives', 'and', 'who', 'declines', ';', 'side', 'factions']
['Can', 'be', 'but', 'brief', ';', 'for', 'I', 'have', 'made', 'him', 'know']
['I', 'am', 'hurt', '.']
